In [22]:
import numpy as np
import torch
import torch.nn.functional as F
from torchvision import transforms
from PIL import Image
from torchvision.utils import save_image
import matplotlib.pyplot as plt
import rasterio
import cv2
import os

In [23]:
tensor2img = transforms.ToPILImage()
img2tensor = transforms.ToTensor()

In [24]:
def chm2binary(chm_img, normalize: bool=True):
    img = chm_img.reshape(chm_img.shape[1], chm_img.shape[2])
    _, thresh = cv2.threshold(img, 5,255,cv2.THRESH_BINARY)
    thresh = thresh.astype(np.uint8)
    return thresh/255 if normalize else thresh

def naip2grayscaled(naip_img, upscaling_dim: tuple=None, grayscale: bool=True):
    if upscaling_dim!=None:
        naip_scaled = cv2.resize(naip_img, upscaling_dim, interpolation = cv2.INTER_CUBIC)
    else:
        naip_scaled = naip_img
    naip_scaled_gray = cv2.cvtColor(naip_scaled, cv2.COLOR_BGR2GRAY)
    return naip_scaled_gray.T/255 if grayscale else naip_scaled.T/255

### Create corresponding gray-scaled naip for binarized chm

In [105]:
def create_data(year, chm:bool=False):
    naip_files = os.listdir(f"./data/naip_CA_{year}_gridded/")
    c=0
    for file in naip_files:
        print(file)
        naip_xy = file.split(".")[0].split("_")
        chm_x = int(naip_xy[0])*2
        chm_y = int(naip_xy[1])*2
        top_left = f"{chm_x}_{chm_y+1}.tiff" 
#         print(top_left)
        top_right = f"{chm_x+1}_{chm_y+1}.tiff" 
#         print(top_right)
        bot_left = f"{chm_x}_{chm_y}.tiff"
#         print(bot_left)
        bot_right = f"{chm_x+1}_{chm_y}.tiff"
#         print(bot_right)
        img1=rasterio.open(f"./data/chm/{top_left}").read()
        img1_dim=img1.shape
#         print(img1_dim)
        img2=rasterio.open(f"./data/chm/{top_right}").read()
        img2_dim=img2.shape
#         print(img2_dim)
        img3=rasterio.open(f"./data/chm/{bot_left}").read()
        img3_dim=img3.shape
#         print(img3_dim)
        img4=rasterio.open(f"./data/chm/{bot_right}").read()
        img4_dim=img4.shape
#         print(img4_dim)
        naip_img=rasterio.open(f"./data/naip_CA_{year}_gridded/{file}").read()[:3]
        naip_dim=naip_img.shape
#         print(naip_dim)
        # Binarize the chm image
        thresh_x1 = torch.tensor(chm2binary(img1))
        thresh_x2 = torch.tensor(chm2binary(img2))
        thresh_x3 = torch.tensor(chm2binary(img3))
        thresh_x4 = torch.tensor(chm2binary(img4))
#         print(thresh_x4)
        X_chm = [thresh_x1.numpy(), thresh_x2.numpy(), thresh_x3.numpy(), thresh_x4.numpy()]
        # Gray-scale the naip
        naip_scaled_gray = torch.tensor(naip2grayscaled(naip_img.T))
        naip_scaled_x1 = naip_scaled_gray[:img1_dim[1],:img1_dim[2]]
        naip_scaled_x2 = naip_scaled_gray[:img2_dim[1], (naip_dim[2]-img2_dim[2]):]
        naip_scaled_x3 = naip_scaled_gray[(naip_dim[1]-img3_dim[1]):, :img3_dim[2]]
        naip_scaled_x4 = naip_scaled_gray[(naip_dim[1]-img4_dim[1]):, (naip_dim[2]-img4_dim[2]):]
        X_naip = [naip_scaled_x1.numpy(), naip_scaled_x2.numpy(), naip_scaled_x3.numpy(), naip_scaled_x4.numpy()]
        # Save images
        output_chm_path = f"./data/target_chm"
        output_naip_path = f"./data/input_naip_{year}"
        if chm:
            if os.path.isdir(output_chm_path)==False:
                os.makedirs(output_chm_path)
            if os.path.isdir(output_naip_path)==False:
                os.makedirs(output_naip_path)
            for x in range(len(X_chm)):
                if X_naip[x].shape[0]/X_naip[x].shape[1]>0.5 and X_naip[x].shape[1]/X_naip[x].shape[0]>0.5:
                    chm_img=Image.fromarray(X_chm[x])
                    naip_img=Image.fromarray(X_naip[x])
                    chm_img.save(f"{output_chm_path}/{c+1}.tiff")
                    naip_img.save(f"{output_naip_path}/{c+1}.tiff")
#                     save_image(X_chm[x], f"{output_chm_path}/{c+1}.tiff")
#                     save_image(X_naip[x], f"{output_naip_path}/{c+1}.tiff")
                    c+=1
        else:
            if os.path.isdir(output_naip_path)==False:
                os.makedirs(output_naip_path)
            for x in range(len(X_naip)):
                if X_naip[x].shape[0]/X_naip[x].shape[1]>0.5 and X_naip[x].shape[1]/X_naip[x].shape[0]>0.5: 
                    naip_img=Image.fromarray(X_naip[x])
                    naip_img.save(f"{output_naip_path}/{c+1}.tiff")
#                     save_image(X_naip[x], f"{output_naip_path}/{c+1}.tiff")
                    c+=1

In [106]:
for year in [2012, 2014, 2016, 2020]:
    if year==2012:
        create_data(year, chm=True)
    else:
        create_data(year)

-12_111.tiff
-8_139.tiff
-13_133.tiff
-11_108.tiff
-11_115.tiff
-8_99.tiff
-7_98.tiff
-11_98.tiff
-13_100.tiff
-8_100.tiff
-12_115.tiff
-11_107.tiff
-13_97.tiff
-9_107.tiff
-13_110.tiff
-9_138.tiff
-11_110.tiff
-13_127.tiff
-6_103.tiff
-6_102.tiff
-10_99.tiff
-9_135.tiff
-13_132.tiff
-6_139.tiff
-11_111.tiff
-13_113.tiff
-13_115.tiff
-13_125.tiff
-7_102.tiff
-12_100.tiff
-13_106.tiff
-8_103.tiff
-10_134.tiff
-6_136.tiff
-12_132.tiff
-8_135.tiff
-8_97.tiff
-12_136.tiff
-9_109.tiff
-10_136.tiff
-13_102.tiff
-12_104.tiff
-12_106.tiff
-9_136.tiff
-10_107.tiff
-9_134.tiff
-6_138.tiff
-8_104.tiff
-11_101.tiff
-9_102.tiff
-7_100.tiff
-13_107.tiff
-11_105.tiff
-13_117.tiff
-9_99.tiff
-7_104.tiff
-7_138.tiff
-11_96.tiff
-6_100.tiff
-12_116.tiff
-11_116.tiff
-12_135.tiff
-8_105.tiff
-13_99.tiff
-11_133.tiff
-8_138.tiff
-10_110.tiff
-10_114.tiff
-13_103.tiff
-12_99.tiff
-10_108.tiff
-11_97.tiff
-10_133.tiff
-9_108.tiff
-12_103.tiff
-10_116.tiff
-12_113.tiff
-12_134.tiff
-10_102.tiff
-9_101.tiff
-

-12_108.tiff
-11_102.tiff
-10_98.tiff
